# Stratégie 2

Ici on s'occupe de charger toutes les bibliothèques nécessaire pour notre étude.

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn as skl
import matplotlib.pyplot as plt

On commence par afficher un premier apperçu de notre dataset, on remarque que le jeu de données contient des informations journalières de 2017 à 2023 sur 8 entreprises du CAC 40. Pour chaque jour, il indique s’il y a eu une actualité concernant chaque entreprise (colonne `X_ACTU`) ou non (valeur "NR"). Lorsqu’une actualité est présente, elle est décrite sous forme de titre. En parallèle, chaque ligne fournit le rendement boursier à 5 jours ouvrés pour chaque entreprise (colonne `X_RDMT_5`). Ainsi, les données permettent d’étudier l’impact potentiel d’une actualité sur le rendement futur d’un actif, en combinant texte (actualités) et données financières (rendements).

In [ ]:
data=pd.read_csv('actu_finance.csv')
data.head()
data.describe()


On va mettre en index les dates et les classer par ordre croissant.


In [ ]:
data['Unnamed: 0']=pd.to_datetime(data['Unnamed: 0'])
data.sort_values('Unnamed: 0')
data.set_index('Unnamed: 0',inplace=True)
data.head()

# Statistique descriptive

On s'occupe ici d'effectuer les statistiques descriptives pour notre dataset.



In [ ]:
data.info()

Le dataset ne contient aucune valeur manquantes. On va maintenat remplacer la valeur "NR" par "nan" et afficher le pourcentage d'actualité disponible pour chaque actif et on affiche des statistiques descriptives génerales.

In [ ]:
data.replace("NR", np.nan, inplace=True)
print("Pourcentages indisponibles par colones")

print(100*data[[col for col in data.columns if col.endswith('_ACTU')]].isna().sum()/data.shape[0])

print("Nombre d'actu indisponibles au total")
print(data.isna().sum().sum())


print("Résumé statistique :")
display(data.describe(exclude=[object]))
data.describe(include=[object])


Ici on sépare notre dataframe d'origine en dataframe pour chaque actif avec l'actualité et le rendement correspondant.

In [ ]:
# dic={}
# for col in data.columns[:8]:
#   if col.endswith('_ACTU'):
#     dic[f"{col.split('_')[0]}"]=data[[f"{col}", f"{col.split('_')[0]}_RDMT_5"]].dropna()

dic={}
for col in data.columns[:8]:
  if col.endswith('_ACTU'):
    dic[f"{col.split('_')[0]}"]=data[[f"{col}", f"{col.split('_')[0]}_RDMT_5"]].dropna()
    dic[f"{col.split('_')[0]}"].columns=['ACTU', 'RDMT']
statistique={}
# for key, value in dic.items():
#   print('Statistiques descriptives pour ',key)
#   print(value.describe(include=[object]))
#   print(value.describe(exclude=[object]))
#   print('-----------------------------------------------------------------------'*2)
#   print('\n')


On affiche ensuite le nombre d'actualité par année

In [ ]:
#dic['BNP']
# rest=pd.concat([dic['BNP'], dic['AIR']])
# rest.dropna().head()

# dic['BNP'].iloc[:,1:2].groupby(dic['BNP'].index.year).count()

# nbr_actu={}
# for key,value in dic.items():
#   print("Nombre d'actualité par année")
#   print(value.iloc[:,1:2].groupby(value.index.year).count())
#   print('-'*100)
#   print('\n')


nbr_actu = {}

for key, value in dic.items():
    # On extrait le nombre d'actualités par année
    counts_by_year = value.iloc[:, 1:2].groupby(value.index.year).count()
    counts_by_year.columns = ['Nombre d\'actualités']  # Optionnel : renommer la colonne

    nbr_actu[key] = counts_by_year

# Fusion de tous les DataFrames en un seul (colonnes multi-indexées par clé)
df_result = pd.concat(nbr_actu, axis=1)

# Affichage
print("Nombre d'actualités par année pour chaque catégorie :")
display(df_result)


On affiche la correlation

In [ ]:

# Select only numerical columns
numerical_cols = data[[x for x in data.columns if x.endswith("_RDMT_5")]]

# Compute the correlation matrix
corr_matrix = numerical_cols.corr()

# Plot the heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap="coolwarm", cbar=True)
plt.title("Correlation Heatmap for Numerical Columns")
plt.show()

On va afficher un graph mentrant le rendement

In [ ]:
# Initialisation du graphique
plt.figure(figsize=(12, 6))
plt.title("Distribution des rendements à 5 jours (RDMT_5) par actif", fontsize=16)
plt.xlabel("Rendement à 5 jours (%)", fontsize=12)
plt.ylabel("Densité", fontsize=12)

# Tracer la courbe KDE pour chaque actif
for key, actif in dic.items():
    sns.kdeplot(data=actif, x="RDMT", label=key, fill=True, alpha=0.3)


# Ajout de la légende
plt.legend(title="Actifs", fontsize=10, title_fontsize=12)
plt.grid(True, linestyle='--', alpha=0.5)
plt.tight_layout()
plt.show()


# Boucle sur chaque actif dans le dictionnaire
for key, df in dic.items():
    plt.figure(figsize=(10, 5))
    sns.kdeplot(data=df, x="RDMT", fill=True, alpha=0.3, color='steelblue')

    plt.title(f"Distribution du rendement à 5 jours pour l'actif {key}", fontsize=15)
    plt.xlabel("Rendement à 5 jours (%)", fontsize=12)
    plt.ylabel("Densité", fontsize=12)
    plt.grid(True, linestyle='--', alpha=0.5)
    plt.tight_layout()
    plt.show()


Pour la suite on va retenir 5 actifs :

In [ ]:
# Liste des actifs choisis — correction des guillemets
choosen_actif = ["AIR", "ORA", "ALO", "CAP", "GLE"]

# Filtrage du dictionnaire avec les actifs choisis
tab = [value for key, value in dic.items() if key in choosen_actif]

# Concaténation des DataFrames sélectionnés
df = pd.concat(tab, axis=0).reset_index()

# Renommage de la colonne index
df.rename(columns={'Unnamed: 0': 'Date'}, inplace=True)

# Définition de l'index temporel
df.set_index('Date', inplace=True)

# Affichage des premières lignes
df.head()


On va calculer le quantile à 20% de la variable RDMT_5 que l'on notera « q20 »
A partir de la variable RDMT_5 construire la variable Y telle que:
- Y = 1 lorsque RDMT_5 < q20
- Y = 0 lorsque RDMT_5 ≥ q20

In [ ]:
q20=df['RDMT'].quantile(0.2)
print(q20)
df['Y']=df['RDMT'].apply(lambda x: 1 if x<q20 else 0)
print(df.head())

In [ ]:
#!pip uninstall torch torchvision torchaudio -y
#!pip install torch torchvision torchaudio


In [ ]:
#!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer

# Chargement du modèle
model = SentenceTransformer("all-MiniLM-L6-v2")

# Encodage de tous les textes en une seule fois (vectorisé)
embeddings = model.encode(df['ACTU'].astype(str).tolist(), show_progress_bar=True)

# Transformation en DataFrame
embeddings_df = pd.DataFrame(embeddings, columns=[f'emb_{i}' for i in range(embeddings.shape[1])])
embeddings_df.set_index(df.index, inplace=True)

# Fusion avec le DataFrame initial
df_encoded = pd.concat([df, embeddings_df], axis=1)



# Visualisation
df_encoded.head()

In [ ]:
df_encoded.head()

In [ ]:

# Extraction des features X et de la cible Y

df_train = df_encoded[df_encoded.index.year < 2023]
df_test = df_encoded[df_encoded.index.year >= 2023]

X_train = df_train.filter(like='emb_')
X_test =  df_test.filter(like='emb_')
y_train = df_train['Y']
y_test =  df_test['Y']



## Neural network

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(1, activation='sigmoid')  # binaire
])

model.compile(optimizer=Adam(0.001), loss='binary_crossentropy', metrics=['accuracy'])
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2)

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob > 0.5).astype(int)

print("F1 Score :", f1_score(y_test, y_pred))
print("Précision :", precision_score(y_test, y_pred))
print("Recall :", recall_score(y_test, y_pred))


In [ ]:
df_encoded.to_csv('actu_finance_emb.csv')

# Modelisation

In [ ]:
!pip install scikeras
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import layers, models
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split



In [ ]:
!pip install scikeras

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# 1️⃣ Fonction pour construire le modèle
def build_model():
    model = Sequential()
    model.add(Dense(128, activation='relu', input_dim=384))
    model.add(Dropout(0.3))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(optimizer='adam',
                  loss='binary_crossentropy',
                  metrics=[tf.metrics.F1Score(average='micro', threshold=0.5),
                  "recall",
                  "precision"
                  ])
    return model

# 2️⃣ Création du pipeline
pipeline = Pipeline([
    ('scaler', StandardScaler()),  # Normalisation des embeddings
    ('clf', KerasClassifier(model=build_model, epochs=20, batch_size=32, verbose=1))
])


 ## Découpage et apprentissage

In [ ]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

# Assure-toi que 'Date' est l'index
df = df_encoded.copy()
df.index = pd.to_datetime(df.index)
df.head()

In [ ]:
# ▪️ Découpe temporelle : train < 2023 / test = 2023
train_df = df[df.index.year < 2023]
test_df = df[df.index.year >= 2023]

# ▪️ Données d'entrée / sortie
X_train = train_df.filter(like='emb_')
y_train = train_df['Y']
X_test = test_df.filter(like='emb_')
y_test = test_df['Y']
X_train.head()

y_train.value_counts()

In [ ]:




# ▪️ Création du MLP
model = Sequential()
model.add(Dense(128, activation='relu', input_dim=X_train.shape[1]))
#model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
#model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

# ▪️ Compilation
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[ "accuracy","recall",
          "precision"])

# ▪️ Entraînement avec EarlyStopping
model.fit(X_train, y_train, validation_split=0.2, epochs=3, batch_size=32)




In [ ]:
# ▪️ Prédiction sur le test
y_pred_prob = model.predict(X_test)
y_pred = (y_pred_prob >= 0.5).astype(int)


# ▪️ Rapport de classification
print(classification_report(y_test, y_pred, digits=4))

In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

print("F1 Score    :", f1_score(y_test, y_pred))
print("Précision   :", precision_score(y_test, y_pred))
print("Recall      :", recall_score(y_test, y_pred))


In [ ]:
df_train['Y'].value_counts()

In [ ]:
from sklearn.utils import class_weight
import numpy as np

# Classe pondérée automatiquement
class_weights = class_weight.compute_class_weight(
    class_weight='balanced',
    classes=np.unique(y_train),
    y=y_train
)
class_weights = dict(enumerate(class_weights))
print("Class weights :", class_weights)


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler

# Standardisation
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Modèle
model = Sequential()
model.add(Dense(128, activation='relu', input_shape=(X_train_scaled.shape[1],)))
model.add(Dropout(0.3))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))

import tensorflow as tf
from tensorflow.keras import backend as K

def f1_metric(y_true, y_pred):
    # Convertir y_true et y_pred au même type
    y_true = K.cast(y_true, 'float32')
    y_pred = K.round(y_pred)

    tp = K.sum(y_true * y_pred)
    fp = K.sum((1 - y_true) * y_pred)
    fn = K.sum(y_true * (1 - y_pred))

    precision = tp / (tp + fp + K.epsilon())
    recall = tp / (tp + fn + K.epsilon())

    f1 = 2 * precision * recall / (precision + recall + K.epsilon())
    return 0.5-f1


# Compilation avec F1 comme métrique
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[f1_metric])

# Early stopping
early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Entraînement
model.fit(X_train_scaled, y_train,
          validation_split=0.2,
          epochs=30,
          batch_size=32,
          class_weight=class_weights,
          verbose=1)


In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

# Prédictions probabilistes
y_pred_prob = model.predict(X_test_scaled)

# Optimisation du seuil
thresholds = np.linspace(0.1, 0.9, 17)
best_thresh, best_f1 = 0.5, 0

for t in thresholds:
    y_pred = (y_pred_prob >= t).astype(int)
    f1 = f1_score(y_test, y_pred)
    if f1 > best_f1:
        best_f1 = f1
        best_thresh = t

print(f"✅ Meilleur seuil : {best_thresh:.2f}, F1 : {best_f1:.4f}")


In [ ]:
y_final = (y_pred_prob >= best_thresh).astype(int)

print("Classification Report :")
from sklearn.metrics import classification_report
print(classification_report(y_test, y_final, digits=4))


In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score

print("F1 Score    :", f1_score(y_test, y_pred))
print("Précision   :", precision_score(y_test, y_pred))
print("Recall      :", recall_score(y_test, y_pred))
